In [1]:
!pip install azure-storage-file-datalake azure-identity pyodbc

DEPRECATION: Loading egg at c:\users\usuario\appdata\local\programs\python\python311\lib\site-packages\nester-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


HAY QUE TENER INSTALADO EN WINDOWS EL DRIVER ODBC PARA BASES DE DATOS SQL QUE SE PUEDE DESCARGAR DE ESTE LINK
https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15#download-for-windows
Igual lo subi al repo de GitHub es el archivo msodbcsql.msi en la carpeta Utils

In [2]:
from azure.identity import InteractiveBrowserCredential

credential = InteractiveBrowserCredential()

In [3]:
import os

In [6]:
from azure.storage.filedatalake import (
    DataLakeServiceClient,
    DataLakeDirectoryClient,
    FileSystemClient
)

In [29]:
#cadena de conexion al punto de datos SQL del datalake de fabric MLModel
sql_endpoint = "nwn7f7ze6vtuxen5age454nhca-p44tzcajmseujpwnyirs57khce.datawarehouse.fabric.microsoft.com" # copy and paste the SQL endpoint from any of the Lakehouses or Warehouses in your Fabric Workspace
database = "MLModel" # copy and paste the name of the Lakehouse or Warehouse you want to connect to
#"https://onelake.dfs.fabric.microsoft.com/883c397f-6409-4489-becd-c2232efd4711/05bfc39e-219f-42de-9013-1ee05672af29/Tables"
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Database=f{database};Encrypt=Yes;TrustServerCertificate=No"

In [8]:
import struct
from itertools import chain, repeat
import pyodbc

In [9]:
# prepare the access token - Aquí se abre una ventana del browser y pide ingresar a nuestra cuenta de fabric para autenticar

token_object = credential.get_token("https://database.windows.net//.default") # Retrieve an access token valid to connect to SQL databases
token_as_bytes = bytes(token_object.token, "UTF-8") # Convert the token to a UTF-8 byte string


In [30]:
encoded_bytes = bytes(chain.from_iterable(zip(token_as_bytes, repeat(0)))) # Encode the bytes to a Windows byte string
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes # Package the token into a bytes object
attrs_before = {1256: token_bytes}  # Attribute pointing to SQL_COPT_SS_ACCESS_TOKEN to pass access token to the driver


# build the connection

connection = pyodbc.connect(connection_string, attrs_before=attrs_before)


In [31]:
cursor = connection.cursor()
cursor.execute("SELECT * FROM sys.tables")
rows = cursor.fetchall()
print(rows) # this will print all the tables available in the lakehouse or warehouse

[('circuitswithimages', 359008360, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5, 53, 317000), datetime.datetime(2024, 9, 3, 12, 5, 53, 317000), False, False, False, 0, None, 10, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False), ('driverswithimages', 375008417, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5, 55, 110000), datetime.datetime(2024, 9, 3, 12, 5, 55, 110000), False, False, False, 0, None, 10, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False), ('constructorswithimages', 391008474, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5,

In [24]:
def generate_dataset(yearfrom,yearuntil,positionfrom,positionuntil):
    cursor.execute( f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,TRY_CAST(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position""")
    df= cursor.fetchall()
    return df

In [25]:
df1= generate_dataset(1982,2024,1,8)

DataError: ('22018', "[22018] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Conversion failed when converting the varchar value '\\N' to data type int. (245) (SQLExecDirectW); [22018] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Statement ID: {3D237F0C-34FE-4BD1-9633-B8049E81A66D} | Query hash: 0x830C1EF2CD8590F2 | Distributed request ID: {7002F2A5-EC97-4736-BF8B-4DE520FF1FA9} (15806)")

In [1]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create my_spark
my_spark=SparkSession.builder.getOrCreate()

# Print my_spark
print(my_spark)


In [3]:
import pandas as pd

In [5]:
positionfrom= 1
positionuntil=3
yearfrom= 1982
yearuntil=2024

In [6]:
query = f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position"""

In [7]:
query

'\n    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,\n    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,\n    re.laps,re.fastestLap,re.fastestLapTime\n    FROM races ra \n    JOIN results re ON ra.raceId = re.raceId\n    JOIN status st ON st.statusId= re.statusId\n    WHERE position >= 1 AND position <= 3 AND ra.year >= 1982 AND ra.year <= 2024 \n    ORDER BY ra.year desc,ra.round, position'

In [20]:
def generate_dataset(yearfrom,yearuntil,positionfrom,positionuntil):
    query = f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position"""
    cursor = conn.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=["raceId","year","round","circuitId","driverId","constructorId","grid","position","points","statusId", "status","laps","fastestLap","fastestLapTime"])
    return df 

In [21]:
df1= generate_dataset(yearfrom,yearuntil,positionfrom,positionuntil)

In [22]:
df1.head(100)

,raceId,year,round,circuitId,driverId,constructorId,grid,position,points,statusId,status,laps,fastestLap,fastestLapTime
0,1121,2024,1,3,830,9,1,1,26,1,Finished,57,39,1:32.608
1,1121,2024,1,3,815,9,5,2,18,1,Finished,57,40,1:34.364
2,1121,2024,1,3,832,6,4,3,15,1,Finished,57,44,1:34.507
3,1121,2024,1,3,840,117,12,10,1,1,Finished,57,30,1:35.632
4,1121,2024,1,3,855,15,17,11,0,11,+1 Lap,56,30,1:35.458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1129,2024,9,7,830,9,2,1,25,1,Finished,70,70,1:15.569
96,1129,2024,9,7,846,1,3,2,18,1,Finished,70,67,1:15.558
97,1129,2024,9,7,847,131,1,3,15,1,Finished,70,70,1:14.895
98,1129,2024,9,7,839,214,18,10,1,1,Finished,70,70,1:17.012


TUTORIAL DE CONEXION DESDE VSCODE AL DATALAKE DE FABRIC A TRAVES DE ODBC:
https://debruyn.dev/2023/connect-to-fabric-lakehouses-warehouses-from-python-code/

In [23]:
def generate_dataset_spark(yearfrom,yearuntil,positionfrom,positionuntil):
    query = f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position"""
    df = my_spark.sql(query)
    return df 

In [26]:
dfspark= generate_dataset_spark(yearfrom,yearuntil,positionfrom,positionuntil)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `races` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 5 pos 9;
'Sort ['ra.year DESC NULLS LAST, 'ra.round ASC NULLS FIRST, 'position ASC NULLS FIRST], true
+- 'Project ['ra.raceId, 'ra.year, 'ra.round, 'ra.circuitId, 're.driverId, 're.constructorId, 're.grid, cast('re.position as int) AS position#0, 're.points, 're.statusId, 'st.status, 're.laps, 're.fastestLap, 're.fastestLapTime]
   +- 'Filter ((('position >= 1) AND ('position <= 3)) AND (('ra.year >= 1982) AND ('ra.year <= 2024)))
      +- 'Join Inner, ('st.statusId = 're.statusId)
         :- 'Join Inner, ('ra.raceId = 're.raceId)
         :  :- 'SubqueryAlias ra
         :  :  +- 'UnresolvedRelation [races], [], fa

In [25]:
dfspark.show()

AttributeError: 'DataFrame' object has no attribute 'show'